In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
import torchvision.datasets as datasets
from torchvision import transforms
from yellowfin import YFOptimizer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class Cnn(nn.Module):
    def __init__(self, dropout=0.5):
        super(Cnn, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d(p=dropout)
        self.fc1 = nn.Linear(1600, 100) # 1600 = number channels * width * height
        self.fc2 = nn.Linear(100, 10)
        self.fc1_drop = nn.Dropout(p=dropout)

    def forward(self, x):
        x = torch.relu(F.max_pool2d(self.conv1(x), 2))
        x = torch.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        
        # flatten over channel, height and width = 1600
        x = x.view(-1, x.size(1) * x.size(2) * x.size(3))
        
        x = torch.relu(self.fc1_drop(self.fc1(x)))
        x = torch.softmax(self.fc2(x), dim=-1)
        return x

In [ ]:
class config:
    def __init__(self):
       self.batch_size = 1000
       self.wd = 0
       self.lr = 0.1
       self.view_every = 128
       self.epochs = 10
       self.mu = 0
       
config = config()
net_yf = Cnn()
net_yf.to(device)
criterion = nn.CrossEntropyLoss()

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()])
#Data
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

#Loaders
trainloader = torch.utils.data.DataLoader(mnist_trainset, batch_size=config.batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(mnist_testset, batch_size=config.batch_size, shuffle=True, num_workers=2)

In [ ]:
optimizer = YFOptimizer(net_yf.parameters(), lr=config.lr, mu=config.mu, weight_decay=config.wd) 

In [ ]:
# Train
for epoch in range(config.epochs):  # loop over the dataset multiple times
    print('EPOCH {:d} / {:d}'.format(epoch + 1, config.epochs))
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net_yf(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % config.view_every == config.view_every-1:
            print('Batch {:d} / {:d}: loss = {:.4f}'.format(i+1,len(trainloader),running_loss/config.view_every))
            running_loss = 0.0

print('Finished Training')

EPOCH 1 / 10


C:\Users\Amine Tourki\Desktop\Master\MA2\optimisation for ML\YellowFin_Pytorch\tuner_utils\yellowfin.py:274: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\python_arg_parser.cpp:1055.)
  state["grad_avg"].mul_(beta).add_(1 - beta, grad)
C:\Users\Amine Tourki\Desktop\Master\MA2\optimisation for ML\YellowFin_Pytorch\tuner_utils\yellowfin.py:555: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self._var_list, self._exploding_grad_clip_target_value + eps)
C:\Users\Amine Tourki\Desktop\Master\MA2\optimisation for ML\YellowFin_Pytorch\tuner_utils\yellowfin.py:526: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(sel

EPOCH 2 / 10
EPOCH 3 / 10
EPOCH 4 / 10
EPOCH 5 / 10
EPOCH 6 / 10
EPOCH 7 / 10
EPOCH 8 / 10
EPOCH 9 / 10
EPOCH 10 / 10
Finished Training


In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network 
        outputs = net_yf(images).to(device)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 94 %
